# Continuous phase-type distribution of branch length two pop two loci

In [ ]:
#library(plotly)
library(ggplot2)
library(viridis)
library(grid)
library(gridExtra)
library(parallel)
library("IRdisplay")
library(repr)
options(repr.plot.width=8, repr.plot.height=5.5)

R header:

In [ ]:
library(tidyverse)
library(devtools)
#remove.packages("ptdalgorithms")
# devtools::install_github("TobiasRoikjer/PtDAlgorithms")
library(ptdalgorithms)

In [ ]:
# %%R
# pkgbuild::check_build_tools(debug = TRUE)

In [ ]:
theme_set(theme_bw())

despine <- theme(panel.border = element_blank(), panel.grid.major = element_blank(),
panel.grid.minor = element_blank(), axis.line = element_line(colour = "black"),
                text=element_text(family="Arial"))

In [ ]:
# p <- ggplot(data=iris, aes(x = Sepal.Length, y = Sepal.Width)) +
#       geom_point(aes(color=Species, shape=Species)) + despine

# ggplotly(width=600, height=400)

In [ ]:
despine <- theme(panel.border = element_blank(), panel.grid.major = element_blank(),
    panel.grid.minor = element_blank(), axis.line = element_line(colour = "black"),
                text=element_text(size=17)) 

options(repr.plot.width=10, repr.plot.height=7)

ggplot(data=iris, aes(x = Sepal.Length, y = Sepal.Width)) +
      geom_point(aes(color=Species, shape=Species), size=5, alpha=0.5) + scale_color_brewer(palette="Set1") + despine

# Utility functions

In [ ]:
# plot_sfs <- function(graph, rewards) {
#     sfs <- sapply(1:(dim(rewards)[1]-1), function(i) expectation(graph, rewards[i,]))
#     data.frame(
#       ton=seq(1,length(sfs)),  
#       brlen=sfs
#       ) %>% ggplot(aes(x=ton, y=brlen, fill=ton)) + 
#       geom_bar(stat = "identity", width=0.8) + scale_fill_viridis() + 
#       despine
# }

# plot_sfs_dph <- function(graph, rewards, trunc=4) {
#     result = data.frame()
#     for (i in 1:(nrow(rewards)-1)) {
#         x <- seq(from = 0, to = trunc, by = 0.01)
#         pdf <- dph(x, reward_transform(graph, rewards[i, ]))
#         df <- data.frame(prob = pdf, t=x, ton=i)
#         result <- rbind(result, df)
#     }
#     result %>% ggplot(aes(y=prob, x=t, group=ton, color=ton)) +
#         geom_line(linewidth=1) + scale_color_viridis() + despine
# }

get_exp_mat <- function(graph, rewards, s) 
{
    exp_mat <- matrix(nrow=s+1,ncol=s+1)
    for (i in 0:s) {
      for (j in 0:s) {
        exp_mat[i+1,j+1] <- expectation(graph, rewards[props_to_index(s, i, j, 1),] + rewards[props_to_index(s, i, j, 2),])
      }
    } 
    return(exp_mat)
}
                                  
plot_exp_mat <- function(exp_mat) 
{  
    df <- as.data.frame(exp_mat) #%>% gather()
    df <- df %>% rownames_to_column('ton1') %>% gather('ton2', 'value', -c(ton1))

    limit <- max(abs(df$value)) * c(-1, 1)
    
    ggplot(df, aes(ton1, ton2)) +
        geom_tile(aes(fill = value)) + 
        geom_text(aes(label = round(value, 2))) +
    scale_x_discrete(labels= seq(0, nrow(exp_mat))) + 
    scale_y_discrete(labels= seq(0, nrow(exp_mat))) + 
    scale_fill_distiller(palette = 'PiYG',direction = 1, limit=limit) +
    theme_minimal() +
     theme(panel.grid.major = element_blank(), 
            panel.grid.minor = element_blank(), 
            text=element_text(size=17))

}

get_cov_mat <- function(graph, rewards) 
{
    n <- nrow(rewards)
    cov_mat <- matrix(nrow=n-1,ncol=n-1)
    for (i in 1:(n-1)) {
        for (j in 1:(n-1)) {
            cov_mat[i, j] <- covariance(graph, rewards[i,], rewards[j,])
        }
    }
    return(cov_mat)
}
                  
plot_cov_mat <- function(cov_mat) 
{
   
    df <- as.data.frame(cov_mat)
    df <- df %>% rownames_to_column('ton1') %>% gather('ton2', 'value', -c(ton1))
    
    ggplot(df, aes(ton1, ton2)) +
        geom_tile(aes(fill = value)) + 
        scale_y_discrete(labels= seq(1, nrow(cov_mat))) + 
        scale_fill_continuous(type = "viridis") + 
        theme_minimal() + 
        theme(panel.grid.major = element_blank(), panel.grid.minor = element_blank(), text=element_text(size=17)) 

}

plot_graph <- function(gam, constrained=TRUE, size=c(6, 6), fontsize=10, rankdir="LR")
{

    if (constrained) {
        constrained <- 'true'
    } else {
        constrained <- 'false'
    }

    edge_templ <- '"FROM" -> "TO" [constraint=true, label="LABEL",labelfloat=false];'
    start_name <- 'IPV'
    absorbing_name <- 'Absorb'
    edges <- c()
    for (i in 1:length(gam$IPV)) {
        if (gam$IPV[i] > 0) {
            edge <- edge_templ
            edge <- sub('FROM', start_name, edge)
            edge <- sub('TO', paste(gam$states[i,], collapse = ","), edge)
            edge <- sub('LABEL', gam$SIM[i, 1], edge)
            edges <- c(edges, edge)
        }
    }    
    for (i in 1:nrow(gam$states)) {
        for (j in 1:nrow(gam$states)) {
            if ((i != j) && (gam$SIM[i, j] > 0)) {
                edge <- edge_templ
                edge <- sub('FROM', paste(gam$states[i,], collapse = ","), edge)
                edge <- sub('TO', paste(gam$states[j,], collapse = ","), edge)
                edge <- sub('LABEL', gam$SIM[i, j], edge)
                edges <- c(edges, edge)
            }
        }
    }
    absorb_rates <- -rowSums(gam$SIM)
    for (i in 1:nrow(gam$states)) {
        if (absorb_rates[i] > 0) {
            # edges <- c(edges, paste('"', from, '"', ' -> ', 'Absorb', '[constraint=true, label="', label, '",labelfloat=false]', ';', sep='')) 
            edge <- edge_templ
            edge <- sub('FROM', paste(gam$states[i,], collapse = ","), edge)
            edge <- sub('TO', absorbing_name, edge)
            edge <- sub('LABEL', absorb_rates[i], edge)
            edges <- c(edges, edge)
        }
    }
    graph_spec <- paste(edges, sep='\n', collapse = '')
    style_str <- '
        rankdir=RANKDIR;
        size="SIZEX,SIZEY";
        fontname="Helvetica,Arial,sans-serif"
    	node [fontname="Helvetica,Arial,sans-serif", fontsize=FONTSIZE]
    	edge [fontname="Helvetica,Arial,sans-serif", fontsize=FONTSIZE]
        Absorb [style=filled,color="lightgrey"]
        IPV [style=filled,color="lightgrey"]
    '
    style_str <- sub('SIZEX', size[1], style_str)
    style_str <- sub('SIZEY', size[2], style_str)
    style_str <- gsub('FONTSIZE', fontsize, style_str)    
    style_str <- gsub('RANKDIR', rankdir, style_str)    
    graph_string <- paste('digraph G {', style_str, graph_spec, '}', sep='\n')
    system("dot -Tsvg -o tmp.svg", input=graph_string, intern=TRUE)
    return(display_svg(file="tmp.svg"))
}

plot_sim <- function(graph)
{
    gam <- graph_as_matrix(graph)
    mat <- t(gam$SIM)
    mat <- matrix(as.integer(mat > 0), dim(mat))
    
    rownames(mat) <- 1:nrow(mat)
    colnames(mat) <- 1:ncol(mat)
    df <- as.data.frame(mat)
    df <- df %>% rownames_to_column('n') %>% gather('m', 'value', -c(n))
    df$n <- as.numeric(df$n)
    df$m <- as.numeric(df$m)
    ggplot(df, aes(n, m)) +
        geom_tile(aes(fill = value), show.legend = FALSE) + 
        scale_y_reverse() +
        scale_fill_gradient(low="white", high="black") +
        theme_minimal() + 
        theme(panel.grid.major = element_blank(), 
              panel.grid.minor = element_blank(), 
              axis.title.x = element_blank(),
              axis.title.y = element_blank(),
              text=element_text(size=17))
}


In [ ]:

make_discrete <- function(mutation_graph, mutation_rate) {
    # Takes a graph for a continuous distribution and turns
    # it into a descrete one (inplace). Returns a matrix of
    # rewards for computing marginal moments

    # current nr of states in graph
    vlength <- vertices_length(mutation_graph)

    # number of fields in state vector (assumes all are the same length)
    state_vector_length <- length(vertex_at(mutation_graph, 1)$state)

    # list state vector fields to reward at each auxiliary node
    rewarded_state_vector_indexes <- vector(mode = "list", length = state_vector_length)

    # loop all but starting node
    for (i in 2:vlength) {
        vertex <- vertex_at(mutation_graph,i)
        if (vertex$rate > 0) { # not absorbing
            for (j in 1:length(vertex$state)) {
                val <- vertex$state[j]
                if (val > 0) { # only ones we may reward
                    # add auxilliary node
                    mutation_vertex <- create_vertex(mutation_graph, rep(0, state_vector_length))
                    add_edge(mutation_vertex, vertex, 1)
                    add_edge(vertex, mutation_vertex, mutation_rate*val)

                    rewarded_state_vector_indexes[[mutation_vertex$index]] <- c(rewarded_state_vector_indexes[[j]], j)
                }
            }
        }
    }
    # normalize graph
    weights_were_multiplied_with <- normalize_graph(mutation_graph)

    # build reward matrix
    rewards <- matrix(nrow=vertices_length(mutation_graph),ncol=state_vector_length, 0)
    for (state in seq_along(rewarded_state_vector_indexes)) {
        for (i in rewarded_state_vector_indexes[[state]]) {
            rewards[state, i] <- 1
        }
    }
    rewards = t(rewards)
    return(rewards)
}

# # self-transition rate:
# mutation_rate <- 0.0001

# # clone graph to get one to modify:
# mutation_graph <- clone_graph(graph)

# # add auxilliary states, normalize and return reward matrix:
# rewards <- make_discrete(mutation_graph, mutation_rate)

# # for plotting the new graph
# gam <- graph_as_matrix(mutation_graph)

# vertices_length(mutation_graph)

# State space

## State space in R

In [ ]:
# r_construct_twolocus_island_graph <- function(s, N, M, R) {

#     # number of populations
#     p <- 2 # needs to be 2
#     # state vector length
#     n <- p*(s+1)**2

#     graph <- create_graph(n)
#     index <- 1
#     # first_vertex <- create_vertex(graph, c(rep(0, s+2), s, rep(0, n-s-3))) # assumes that p=2
#     state <- rep(0, n)
#     state[conf_to_index(s, 1, 1, 1)] <- s
#     first_vertex <- create_vertex(graph, state) # assumes that p=2
#     add_edge(starting_vertex(graph), first_vertex, 1)

#     index <- 2
#     while (index <= vertices_length(graph)) {

#       vertex <- vertex_at(graph, index)
#       state <- vertex$state

#       count <- 0
#       for (i in 1:n) {
#           count <- count + state[i]
#       }
#       if (count <= 1) {
#           # Only one lineage, stop
#           index <- index + 1
#           next
#       }    

#       for (i in 1:n) {
#         conf_i <- index_to_conf(s, i)

#         # coalescence #########################
#         for (j in i:n) {
#           conf_j <- index_to_conf(s, j)

#           if (conf_i@population != conf_j@population) {
#             # different populations
#             next
#           }
#           if (i == j) {
#             if (state[i] < 2) {
#               next;
#             }
#             rate <- state[i] * (state[i] - 1) / 2 / N
#           } else {
#             if (state[i] < 1 || state[j] < 1) {
#               next;
#             }
#             rate <- state[i] * state[j] / N
#           }

#           child_state <- state

#           # lineages with index i and j coalesce:  
#           child_state[i] <- child_state[i] - 1
#           child_state[j] <- child_state[j] - 1
#           stopifnot(conf_i@locus1+conf_j@locus1 <= s)
#           stopifnot(conf_i@locus2+conf_j@locus2 <= s)

#           # coalescene into lineage with index k
#           k = conf_to_index(s, conf_i@locus1+conf_j@locus1, conf_i@locus2+conf_j@locus2, conf_i@population)
#           child_state[k] <- child_state[k] + 1

#           child_vertex <- find_or_create_vertex(graph, child_state)
#           add_edge(vertex, child_vertex, rate)
#         }

#         # recombination #######################
#         if (state[i] > 0 && conf_i@locus1 > 0 && conf_i@locus2 > 0) {

#           rate <- R
#           child_state <- state

#           # a lineage with index i recombines to produce lineages with index k and l
#           k = conf_to_index(s, conf_i@locus1, 0, conf_i@population)
#           l = conf_to_index(s, 0, conf_i@locus2, conf_i@population)
#           child_state[i] <- child_state[i] - 1
#           child_state[k] <- child_state[k] + 1
#           child_state[l] <- child_state[l] + 1

#           child_vertex <- find_or_create_vertex(graph, child_state)
#           add_edge(vertex, child_vertex, rate)
#         }

#         # migration ###########################
#         if (state[i] > 0) {

#           rate <- M
#           child_state <- state

#           if (conf_i@population == 1) {
#             m = 2
#           } else {
#             m = 1
#           }
#           # 
#           k = conf_to_index(s, conf_i@locus1, conf_i@locus2, m)
#           child_state[i] <- child_state[i] - 1
#           child_state[k] <- child_state[k] + 1

#           child_vertex <- find_or_create_vertex(graph, child_state)
#           add_edge(vertex, child_vertex, rate)
#         }
#       }

#       index <- index + 1

#       if ((index %% 50) == 0) {
#         cat(index, vertices_length(graph), "\n")
#       }

#     }
    
#     return(graph)
# }

# graph <- r_construct_twolocus_island_graph(2, 1, 0, 1)

# states <- t(sapply(1:vertices_length(graph), function(index) vertex_at(graph, index)$state, simplify = FALSE))
# ipv <- graph_as_matrix(graph)$IPV
# sim <- graph_as_matrix(graph)$SIM

# graph_as_matrix(graph)

## State space in C

In [ ]:
Rcpp::sourceCpp("./cpp/two_locus_two_island.cpp")

Zero-probability transistions are ignored, so the state space is reduced when either recombination rate or migration rate are zero.

Each state has the following three properties:
1. nr decendants at locus 1
2. nr decendants at locus 2
3. what population the lineage is currently in

Rccp exports `construct_twolocus_island_graph`, `index_to_props`, and `props_to_index`:

Sample size:

In [ ]:
s <- 2

# Sanity checks without migration

## ARG TMRCA without migration

The minimum of two independent exponential random variables (the loci are independent with large recombination rate) with parameters $\lambda$ and $\eta$ is also exponential with parameter $\lambda+\eta$. Also:

$$\mathbb{E}[\min(X_1, X_2) + \max(X_1, X_2)] = \mathbb{E}[X_1 + X_2] = \frac{1}{\lambda} + \frac{1}{\eta}$$

Because 

$$\mathbb{E}[\min(X_1, X_2)] = \frac{1}{\lambda + \eta},$$

we get

$$\mathbb{E}[\min(X_1, X_2)] = \frac{1}{\lambda} + \frac{1}{\eta} - \frac{1}{\lambda + \eta}$$

So for a sample of two, and a coalescence rate of 1, the expected time to coalescence at *both* loci is $1 + 1 - \frac{1}{1+1} = 1.5$. The time for the two loci to coalesce is 1. So the total ARG TMRCA should be 2.5:

In [ ]:

graph <- construct_twolocus_island_graph(2, 1, 1, 0, 0, 0)
expectation(graph)

In [ ]:
graph <- construct_twolocus_island_graph(2, 1, 1, 0, 0, 10000000)
expectation(graph)

__*Note for Tobias*__: `expectation` does not like null-edges:

In [ ]:
# graph <- construct_twolocus_island_graph(2, 1, 1, 0, 0, 10000000, keep_null_edges=TRUE)
# expectation(graph)

## Visualize graph

In [ ]:
gam <- graph_as_matrix(graph)

In [ ]:
plot_graph(gam)

In [ ]:
index_to_props(s, 4)

## Two-locus SFS without recombination

Expected ARG branches with i and j tons at each locus

In [ ]:
s <- 5

graph <- construct_twolocus_island_graph(s, 1, 1, 0, 0, 0)
rewards <- sapply(1:vertices_length(graph), function(index) vertex_at(graph, index)$state )
print(vertices_length(graph))

In [ ]:
plot_sim(graph)

In [ ]:
exp_mat <- matrix(nrow=s+1,ncol=s+1)
for (i in 1:s) {
  for (j in 1:s) {
    exp_mat[i+1,j+1] <- expectation(graph, rewards[props_to_index(s, i, j, 1),] + rewards[props_to_index(s, i, j, 2),])
  }
}

In [ ]:
nrow(rewards)

In [ ]:
rewards

In [ ]:
plot_exp_mat(get_exp_mat(graph, rewards, s))

## Two-locus SFS with recombination

Expected ARG branches with i and j tons at each locus

In [ ]:
s <- 4
graph <- construct_twolocus_island_graph(s, 1, 1, 0, 0, 5)
rewards <- sapply(1:vertices_length(graph), function(index) vertex_at(graph, index)$state )
expectation(graph)
print(vertices_length(graph))

In [ ]:
plot_sim(graph)

In [ ]:
exp_mat <- get_exp_mat(graph, rewards, s)

In [ ]:
plot_exp_mat(exp_mat)

Marginal SFSs:

In [ ]:
x <- colSums(exp_mat)
sfs_locus1 <- x[c(-1, -length(x))]
x <- rowSums(exp_mat)
sfs_locus2 <- x[c(-1, -length(x))]

Fits expectations for a single locus:

In [ ]:
grid.arrange(
    data.frame(
      ton=seq(1,length(sfs_locus1)),  
      brlen=sfs_locus1
      ) %>% ggplot(aes(y=brlen, x=ton, fill=ton)) + 
            geom_bar(stat = "identity", width=0.8) + 
            scale_fill_viridis() + 
            despine + 
            theme(legend.position = "none"),
    data.frame(
      ton=seq(1,length(sfs_locus2)),  
      brlen=sfs_locus2
      ) %>% ggplot(aes(y=brlen, x=ton, fill=ton)) + 
            geom_bar(stat = "identity", width=0.8) + 
            scale_fill_viridis() + 
            despine + 
            theme(legend.position = "none"),
    nrow = 1)


# PDFs of ARG branch length with i and j descendants at each locus

In [ ]:
result = data.frame()
for (i in 1:(s-1)) {
    for (j in 1:(s-1)) {
        x <- seq(from = 0, to = 4, by = 0.01)
        pdf <- dph(x, reward_transform(graph, rewards[props_to_index(s, i, j, 1),]))

        df <- data.frame(prob = pdf, t=x, i_ton=i, j_ton=j, color=100*j+i)
        result <- rbind(result, df)
    }
}
#result %>% ggplot(aes(y=prob, x=t, color=ton, group=ton)) + geom_line(size=1)

In [ ]:
ggplot(result, aes(y=prob, x=t)) + geom_line(linewidth=2, color='#238A8DFF') +
    facet_grid(i_ton ~ j_ton) +
    theme_bw() + theme(panel.grid.major = element_blank(), panel.grid.minor = element_blank(), strip.background = element_rect(fill="white"))
ggsave('~/Desktop/tmp.pdf', width=7, height=6)

# ARG TMRCA without recombination

Test the classical result that with symmetric migration and sampling from one population, the expectation should be the sum of the two population sizes (2) and independent of the migration rate. The variance is not and should fall with higher migration rate and asymptotically reach (4):

In [ ]:
graph <- construct_twolocus_island_graph(2, 1, 1, 10000000000, 10000000000, 0)
variance(graph)

In [ ]:
expectation_migrates <- c()
variance_migrates <- c()
mig_rates <- 10^(-5:5)
for (mig_rate in mig_rates) {
    graph <- construct_twolocus_island_graph(2, 1, 1, mig_rate, mig_rate, 0)
    expectation_migrates <- c(expectation_migrates, expectation(graph))
    variance_migrates <- c(variance_migrates, variance(graph))   
}

In [ ]:
data.frame(mig_rates=mig_rates, expectation_migrates=expectation_migrates)

In [ ]:
ggplot(data.frame(mig_rates=mig_rates, 
                         expectation_migrates=expectation_migrates),
             aes(x=mig_rates, expectation_migrates)) +
    geom_line() +
    geom_line(data=data.frame(mig_rates=mig_rates, 
                         variance_migrates=variance_migrates),
             aes(x=mig_rates, variance_migrates)) +
        scale_x_continuous(trans='log10') + 
        scale_y_continuous(trans='log10') +
        geom_hline(yintercept = 2, color='blue', linetype='dashed') +
        geom_hline(yintercept = 4, color='red', linetype='dashed') + despine

# Distribution of number of outgoing edes

In [ ]:
s <- 5

In [ ]:
graph <- construct_twolocus_island_graph(s, 1, 1, 0, 0, 1)
print(vertices_length(graph))
si <- sign(graph_as_matrix(graph)$SIM)
diag(si) <- 0
outgoing_edge_counts <- rowSums(si)
# qplot(outgoing_edge_counts)

In [ ]:
qplot(outgoing_edge_counts, bins=10)

# Matrix sparcity

In [ ]:
gam <- graph_as_matrix(graph)
mean(as.integer(gam$SIM > 0))

In [ ]:
plot_sim(graph)
ggsave('~/Desktop/tmp.pdf')

# Two-locus SFS

Expected ARG branches with i and j tons at each locus

In [ ]:
# %%R -o s
# s <- 4

No migration:

In [ ]:
# %%R
# graph <- construct_twolocus_island_graph(s, 1, 1, 1, 1, 1)
# rewards <- sapply(1:vertices_length(graph), function(index) vertex_at(graph, index)$state )
# vertices_length(graph)

In [ ]:
# %%R

# library(foreach)
# foreach (i=1:3) %do% {
#   sqrt(i)
# }

In [ ]:
broadcast_props_to_index <- function(s, a, b, p) {
    return((p-1)*(s + 1)**2 + a*(s + 1) + b + 1)
}

two_locus_expectation <- function(s, N, M, R) 
{
    graph <- construct_twolocus_island_graph(s, N, N, M, M, R)
    rewards <- sapply(1:vertices_length(graph), function(index) vertex_at(graph, index)$state )

    exp_mat <- matrix(nrow=s+1,ncol=s+1)
    for (i in 0:s) {
      for (j in 0:s) {
        exp_mat[i+1,j+1] <- expectation(graph, rewards[props_to_index(s, i, j, 1),] + rewards[props_to_index(s, i, j, 2),])
      }
    }
    return(exp_mat)
} 

In [ ]:
exp_mat <- two_locus_expectation(s, 1, 1, 1)
exp_mat

In [ ]:
plot_exp_mat(exp_mat) 

In [ ]:
df <- as.data.frame(exp_mat) #%>% gather()
df <- df %>% rownames_to_column('ton1') %>% gather('ton2', 'value', -c(ton1))

ggplot(df, aes(ton1, ton2)) +
    geom_tile(aes(fill = value)) + 
scale_x_discrete(labels= seq(0, nrow(exp_mat))) + 
scale_y_discrete(labels= seq(0, nrow(exp_mat))) + 
scale_fill_viridis() +
theme_minimal() +
theme(aspect=1) + labs(y = "locus 2", x = "locus 1", fill='expectation') +
 theme(panel.grid.major = element_blank(), 
        panel.grid.minor = element_blank(), 
        text=element_text(size=17))

ggsave('~/Desktop/tmp.pdf', width=6, height=6)

# Covriance between tons at locus one

In [ ]:
broadcast_props_to_index <- function(s, a, b, p) {
    return((p-1)*(s + 1)**2 + a*(s + 1) + b + 1)
}

ton_covariance_within_locus <- function(s, N, M, R) 
{
    graph <- construct_twolocus_island_graph(s, N, N, M, M, R)
    rewards <- sapply(1:vertices_length(graph), function(index) vertex_at(graph, index)$state )

    locus1_rewards <- function(i) {
        colSums(rewards[broadcast_props_to_index(s, i, 1:s, 1),]) + colSums(rewards[broadcast_props_to_index(s, i, 1:s, 2),])
    }

    fun <- function(a, b) mapply(function(i, j) covariance(graph, locus1_rewards(i), locus1_rewards(j)), a, b)
    cov_mat <- outer(1:(s-1), 1:(s-1), fun)
    return(cov_mat)
}    

No migration and no recombination:

In [ ]:
cov_mat_no_mig <- ton_covariance_within_locus(s, 1, 0, 0) 

In [ ]:
plot_cov_mat(cov_mat_no_mig)

In [ ]:
plot_cov_mat(cov_mat_no_mig) + theme(aspect=1) + labs(y = "nr descendants", x = "nr descendants", fill='covariance')
ggsave('~/Desktop/tmp.pdf', width=4.7, height=4.7)

# Covriance between tons between loci

In [ ]:
broadcast_props_to_index <- function(s, a, b, p) {
    return((p-1)*(s + 1)**2 + a*(s + 1) + b + 1)
}

ton_covariance_between_loci <- function(s, N, M, R) 
{
    graph <- construct_twolocus_island_graph(s, N, N, M, M, R)
    rewards <- sapply(1:vertices_length(graph), function(index) vertex_at(graph, index)$state )

    locus1_rewards <- function(i) {
        colSums(rewards[broadcast_props_to_index(s, i, 1:s, 1),]) + colSums(rewards[broadcast_props_to_index(s, i, 1:s, 2),])
    }
    locus2_rewards <- function(j) {
        colSums(rewards[broadcast_props_to_index(s, 1:s, j, 1),]) + colSums(rewards[broadcast_props_to_index(s, 1:s, j, 2),])
    }

    fun <- function(a, b) mapply(function(i, j) covariance(graph, locus1_rewards(i), locus2_rewards(j)), a, b)
    cov_mat <- outer(1:(s-1), 1:(s-1), fun)
    return(cov_mat)
}    

No migration and no recombination:

In [ ]:
cov_mat_no_rec_no_mig <- ton_covariance_between_loci(s, 1, 0, 0) 

In [ ]:
plot_cov_mat(cov_mat_no_rec_no_mig) + theme(aspect=0.95) + labs(y = "nr descendants", x = "nr descendants", fill='covariance')

With recombination:

In [ ]:
cov_mat_no_mig <- ton_covariance_between_loci(s, 1, 0, 1)
plot_cov_mat(cov_mat_no_mig) + theme(aspect=0.95) + labs(y = "nr descendants", x = "nr descendants", fill='covariance')
ggsave('~/Desktop/tmp.pdf', width=4.7, height=4.7)

In [ ]:
cov_mat_low_rec <- ton_covariance_between_loci(s, 1, 0, 0.1) 
plot_cov_mat(cov_mat_low_rec) + theme(aspect=0.95) + labs(y = "nr descendants", x = "nr descendants", fill='covariance')

In [ ]:
cov_mat_mig <- ton_covariance_between_loci(s, 1, 0.1, 1) 
plot_cov_mat(cov_mat_mig)  + theme(aspect=0.95) + labs(y = "nr descendants", x = "nr descendants", fill='covariance')

# Two locus two islands

# Covariance of ARG branches in the two populations

Covariance of two-locus "ton" branch length (e.g. singletons at locus 1 and doubletons at locus 2) spent in each population.

In [ ]:
graph <- construct_twolocus_island_graph(s, 1, 1, 1, 1, 1)
rewards <- sapply(1:vertices_length(graph), function(index) vertex_at(graph, index)$state )

In [ ]:
fun <- function(a, b) {
    mapply(function(i, j) covariance(graph, 
                                     rewards[props_to_index(s, i, j, 1),], 
                                     rewards[props_to_index(s, i, j, 2),]), 
           a, b)
}
cov_mat <- outer(1:(s-1), 1:(s-1), fun)

In [ ]:
plot_cov_mat(cov_mat) + theme(aspect=0.95) + labs(y = "nr descendants", x = "nr descendants", fill='covariance')
ggsave('~/Desktop/tmp.pdf', width=4.7, height=4.7)

Shows that when there are hte same number of decendants at each locus (mostly becuase of no recombination) then the covariance is highest. Mostly for small tons where there are many lineages to back and forth between popoulations.

In [ ]:
# %%R

# setClass("conf", slots=list(locus1="numeric", locus2="numeric", population="numeric"))

# obj <- new("conf", locus1=3, locus2=2, population=1)
# obj@population

# r_index_to_props <- function(s, i) {
#     #' returns the locus1, locus2, population
#     #' conformation of lineages represented by
#     #' state vector index i.
#     stopifnot(i > 0)
#     d <- s + 1 # dim
#     idx <- i - 1
#     p <- idx %/% d**2
#     stopifnot(i <= 2*d**2)
#     a <- (idx - p*d**2) %/% d
#     b <- (idx - p*d**2) %% d
#     return(new("conf", locus1=a, locus2=b, population=p+1))
# }
# r_props_to_index <- function(s, a, b, p) {
#     #' returns the state vector index representing
#     #' the locus1, locus2, population conformation.
#     d <- s + 1
#     i = (p-1)*d**2 + a*d + b + 1
#     return(i)
# }

# Fit landscapes

## Discrete versions

In [ ]:
# disc_two_locus_expectation <- function(s, N, M, R) 
# {    
#     graph <- construct_twolocus_island_graph(s, N, N, M, M, R)

#     mutation_graph <- clone_graph(graph)
#     rewards <- make_discrete(mutation_graph, mutation_rate)
    
#     exp_mat <- matrix(nrow=s+1,ncol=s+1)
#     for (i in 0:s) {
#       for (j in 0:s) {
#         exp_mat[i+1,j+1] <- expectation(mutation_graph, rewards[props_to_index(s, i, j, 1),] + rewards[props_to_index(s, i, j, 2),])
#       }
#     }
#     return(exp_mat)
# } 


# self-transition rate:
mutation_rate <- 0.0001

disc_two_locus_expectation <- function(s, N, M, R) 
{    
    graph <- construct_twolocus_island_graph(s, N, N, M, M, R)

    mutation_graph <- clone_graph(graph)
    rewards <- make_discrete(mutation_graph, mutation_rate)
    
    exp_mat <- matrix(nrow=s-1,ncol=s-1)
    for (i in 1:(s-1)) {
      for (j in 1:(s-1)) {
        exp_mat[i,j] <- expectation(mutation_graph, rewards[props_to_index(s, i, j, 1),] + rewards[props_to_index(s, i, j, 2),])
      }
    }
    return(exp_mat)
} 

disc_ton_covariance_within_locus <- function(s, N, M, R) 
{
    graph <- construct_twolocus_island_graph(s, N, N, M, M, R)
    
    mutation_graph <- clone_graph(graph)
    rewards <- make_discrete(mutation_graph, mutation_rate)
    
    locus1_rewards <- function(i) {
        colSums(rewards[broadcast_props_to_index(s, i, 1:s, 1),]) + colSums(rewards[broadcast_props_to_index(s, i, 1:s, 2),])
    }

    fun <- function(a, b) mapply(function(i, j) covariance(mutation_graph, locus1_rewards(i), locus1_rewards(j)), a, b)
    cov_mat <- outer(1:(s-1), 1:(s-1), fun)
    return(cov_mat)
}  

disc_ton_covariance_between_loci <- function(s, N, M, R) 
{
    graph <- construct_twolocus_island_graph(s, N, N, M, M, R)

    mutation_graph <- clone_graph(graph)
    rewards <- make_discrete(mutation_graph, 1)
    
    locus1_rewards <- function(i) {
        colSums(rewards[broadcast_props_to_index(s, i, 1:s, 1),]) + colSums(rewards[broadcast_props_to_index(s, i, 1:s, 2),])
    }
    locus2_rewards <- function(j) {
        colSums(rewards[broadcast_props_to_index(s, 1:s, j, 1),]) + colSums(rewards[broadcast_props_to_index(s, 1:s, j, 2),])
    }

    fun <- function(a, b) mapply(function(i, j) covariance(mutation_graph, locus1_rewards(i), locus2_rewards(j)), a, b)
    cov_mat <- outer(1:(s-1), 1:(s-1), fun)
    return(cov_mat)
} 

In [ ]:
mat <- disc_two_locus_expectation(5, 1, 1, 1)
plot_exp_mat(mat) +
scale_x_discrete(labels= seq(0, nrow(exp_mat))) + 
scale_y_discrete(labels= seq(0, nrow(exp_mat))) + 
scale_fill_viridis() +
theme_minimal() +
theme(aspect=1) + labs(y = "locus 2", x = "locus 1", fill='expectation') +
 theme(panel.grid.major = element_blank(), 
        panel.grid.minor = element_blank(), 
        text=element_text(size=17))

ggsave('~/Desktop/tmp.pdf', width=6, height=6)

mat <- disc_two_locus_expectation(5, 1, 1, 1)
df <- as.data.frame(mat) #%>% gather()
df <- df %>% rownames_to_column('ton1') %>% gather('ton2', 'value', -c(ton1))
ggplot(df, aes(ton1, ton2)) +
    geom_tile(aes(fill = value)) + 
scale_x_discrete(labels= seq(0, nrow(exp_mat))) + 
scale_y_discrete(labels= seq(0, nrow(exp_mat))) + 
scale_fill_viridis() +
theme_minimal() +
theme(aspect=1) + labs(y = "locus 2", x = "locus 1", fill='expectation') +
 theme(panel.grid.major = element_blank(), 
        panel.grid.minor = element_blank(), 
        text=element_text(size=17))

ggsave('~/Desktop/tmp.pdf', width=6, height=6)

In [ ]:

N <- 1
for (M in c(0.1, 1, 10)) {
    for (R in c(0.1, 1, 10)) {
        file_name <- paste('~/Desktop/tmp_M=', M, '_R=', R, '.pdf', sep='')
    
        mat <- disc_ton_covariance_between_loci(5, N, M, R)
        plot_cov_mat(mat) + theme(aspect=0.95) + labs(y = "locus 2", x = "locus 1", fill='covariance')
        ggsave(file_name, width=4.7, height=4.7)
    }
}

In [ ]:
N <- 1
M <- 0.01

for (R in c(0.1, 1, 10)) {
    file_name <- paste('~/Desktop/tmp_M=', M, '_R=', R, '.pdf', sep='')

    mat <- disc_ton_covariance_between_loci(5, N, M, R)
    plot_cov_mat(mat) + theme(aspect=0.95) + labs(y = "locus 2", x = "locus 1", fill='covariance')
    ggsave(file_name, width=4.7, height=4.7)
}

N <- 1
M <- 0.1

for (R in c(0.1, 1, 10)) {
    file_name <- paste('~/Desktop/tmp_M=', M, '_R=', R, '.pdf', sep='')

    mat <- disc_ton_covariance_between_loci(5, N, M, R)
    plot_cov_mat(mat) + theme(aspect=0.95) + labs(y = "locus 2", x = "locus 1", fill='covariance')
    ggsave(file_name, width=4.7, height=4.7)
}

In [ ]:
N <- 1
R <- 1

for (M in c(0.01, 0.1, 1)) {
    file_name <- paste('~/Desktop/tmp_M=', M, '_R=', R, '.pdf', sep='')
    mat <- disc_ton_covariance_between_loci(5, N, M, R)
    plot_cov_mat(mat) + theme(aspect=0.95) + labs(y = "locus 2", x = "locus 1", fill='covariance')
    ggsave(file_name, width=4.7, height=4.7)
}

In [ ]:
#disc_cov_mat_no_rec_no_mig <- disc_ton_covariance_between_loci(5, 1, 1, 0.5, 0.5, 20) 

## Single core

In [ ]:
# s <- 5

# expected_cov_mat <- ton_covariance_between_loci(s, 1, 0.5, 2)
# # expected_cov_mat <- (expected_cov_mat - mean(expected_cov_mat)) / sd(expected_cov_mat)

# expected_exp_mat <- two_locus_expectation(s, 1, 0.5, 2)
# # expected_exp_mat <- (expected_exp_mat - mean(expected_exp_mat)) / sd(expected_exp_mat)


# fit <- function(N, M) {
#     observed_cov_mat <- ton_covariance_between_loci(s, N, M, 2)
#     # observed_cov_mat <- (observed_cov_mat - mean(observed_cov_mat)) / sd(observed_cov_mat)
    
#     observed_exp_mat <- two_locus_expectation(s, N, M, 2)
#     # observed_exp_mat <- (observed_exp_mat - mean(observed_exp_mat)) / sd(observed_exp_mat)
    
#     cov_fit <- abs(sum((observed_cov_mat - expected_cov_mat) / expected_cov_mat, na.rm=TRUE))
#     exp_fit <- abs(sum((observed_exp_mat - expected_exp_mat) / expected_exp_mat, na.rm=TRUE))
#     return(exp_fit + cov_fit)
# }

# n <- seq(0.5, 1.5, 0.05)
# m <- seq(0.1, 1.1, 0.05)
# params <- expand.grid(n, m)
# fit_mat <- matrix(mapply(fit, params$Var1, params$Var2), nrow=length(n),ncol=length(m))

# rownames(fit_mat) <- n
# colnames(fit_mat) <- m

# df <- as.data.frame(fit_mat) #%>% gather()
# df <- df %>% rownames_to_column('n') %>% gather('m', 'value', -c(n))
# df$n <- as.numeric(df$n)
# df$m <- as.numeric(df$m)

# ggplot(df, aes(n, m)) +
#     geom_tile(aes(fill = value)) + 
#     geom_text(aes(label = round(value, 1))) +
#         scale_fill_continuous(type = "viridis") + 
#         theme_minimal() + 
#         theme(panel.grid.major = element_blank(), 
#               panel.grid.minor = element_blank(), 
#               text=element_text(size=17))

## Multiprocessing

Experiemnts with the fit surface when fitting

- The expectation (the standard SFS)
- The covariance matrix for pairs of tons at the same locus (This cannot be obtained emprically from data - or can it)
- The covariance matrix for pairs of tons at loci offset by some genetic distance.

In [ ]:
rmsd <- function(a, b)
{
    stopifnot(dim(a) == dim(b), nrow(a) == ncol(a))
    return( sqrt(sum((a - b)^2)/(dim(a)[1]^2)) )
}

apply_grid <- function(fun, a, b)
{
    named_params <- expand.grid(a, b)
    param_list <- split(named_params, seq(nrow(named_params)))    
    result_list <- mclapply(param_list, function(x) do.call(fit, as.list(as.vector(unlist(x)))), mc.cores=4L)
    fit_mat <- matrix(unlist(result_list), nrow=length(a),ncol=length(b))
    return(fit_mat)
}

plot_fit_mat <- function(fit_mat, labels=TRUE)
{
    df <- as.data.frame(fit_mat)
    df <- df %>% rownames_to_column('n') %>% gather('m', 'value', -c(n))
    df$n <- as.numeric(df$n)
    df$m <- as.numeric(df$m)
    
    p <- ggplot(df, aes(n, m)) +
        geom_tile(aes(fill = value)) + 
            scale_fill_continuous(type = "viridis", direction=-1) + 
            theme_minimal() + 
            theme(panel.grid.major = element_blank(), 
                  panel.grid.minor = element_blank(), 
                  text=element_text(size=17))
    if (labels) {
        p <- p + geom_text(aes(label = round(value, 1)), size=3) 
    }
    return(p)
}

In [ ]:
s <- 5
n <- seq(0.5, 1.5, 0.05)
m <- seq(0.1, 1.1, 0.05)
# m <- seq(0.01, 0.11, 0.005) 

----

A discrete version of one of the ones below:

In [ ]:
# R <- 20
# fit <- function(N, M)
# {
#     (rmsd(disc_ton_covariance_within_locus(s, N, M, R), within_cov_mat) +
#      rmsd(disc_ton_covariance_between_loci(s, N, M, R), between_cov_mat) +
#      rmsd(disc_two_locus_expectation(s, N, M, R), exp_mat)
#     ) / 3
# }
# within_cov_mat <- disc_ton_covariance_within_locus(s, 1, 0.5, R)
# between_cov_mat <- disc_ton_covariance_between_loci(s, 1, 0.5, R)
# exp_mat <- disc_two_locus_expectation(s, 1, 0.5, R)
# fit_mat <- apply_grid(fit, n, m)
# rownames(fit_mat) <- n
# colnames(fit_mat) <- m

In [ ]:
# df <- as.data.frame(-log(fit_mat), 10)
# df <- df %>% rownames_to_column('n') %>% gather('m', 'value', -c(n))
# df$n <- as.numeric(df$n)
# df$m <- as.numeric(df$m)
# ggplot(df, aes(n, m)) +
#     geom_tile(aes(fill = value)) + 
#     geom_text(aes(label = round(value, 1)), size=3) +
#         scale_fill_continuous(type = "viridis", limits = c(1,30), direction=-1) + 
#         theme_minimal() + 
#         theme(panel.grid.major = element_blank(), 
#               panel.grid.minor = element_blank(), 
#               text=element_text(size=17))

In [ ]:
# R <- 2
# fit <- function(N, M)
# {
#     abs(sum((disc_ton_covariance_between_loci(s, N, M, R) - between_cov_mat) / between_cov_mat, na.rm=TRUE)) +   
#     abs(sum((disc_two_locus_expectation(s, N, M, R) - exp_mat) / exp_mat, na.rm=TRUE))
# }
# between_cov_mat <- disc_ton_covariance_between_loci(s, 1, 0.5, R)
# exp_mat <- disc_two_locus_expectation(s, 1, 0.5, R)
# fit_mat <- apply_grid(fit, n, m)
# rownames(fit_mat) <- n
# colnames(fit_mat) <- m
# plot_fit_mat(fit_mat)

In [ ]:
# graph <- construct_twolocus_island_graph(2, 1, 1, 1, 1, 1)
# plot_sim(graph)

---

In [ ]:
s <- 5

# true_N <- 1
# true_M <- 0.5
# n <- seq(0.5, 1.5, 0.05)
# m <- seq(0.1, 1.1, 0.05)

true_N <- 1
true_M <- 0.05
n <- seq(0.5, 1.5, 0.05)
m <- seq(0.01, 0.101, 0.005)

In [ ]:
R <- 1
fit <- function(N, M) {
    exp_fit <- rmsd(two_locus_expectation(s, N, M, R), expected_exp_mat)
    return(exp_fit)
}
expected_exp_mat <- two_locus_expectation(s, true_N, true_M, R)
fit_mat <- apply_grid(fit, n, m)
rownames(fit_mat) <- n
colnames(fit_mat) <- m

plot_fit_mat(fit_mat, labels=FALSE) + geom_point(data = data.frame(n=c(true_N), m=(true_M)), color='red') + labs(y='N', x='N', fill="RMSD") + theme(aspect=1)
ggsave('~/Desktop/expectation_fit.pdf', width=6, height=6)

In [ ]:
R <- 1
fit <- function(N, M) {
    within_cov_fit <- rmsd(ton_covariance_within_locus(s, N, M, R), expected_within_cov_mat)
    return(within_cov_fit)
}
expected_within_cov_mat <- ton_covariance_within_locus(s, true_N, true_M, R)
fit_mat <- apply_grid(fit, n, m)
rownames(fit_mat) <- n
colnames(fit_mat) <- m
plot_fit_mat(fit_mat, labels=FALSE) + geom_point(data = data.frame(n=c(true_N), m=(true_M)), color='red') + labs(y='N', x='N', fill="RMSD") + theme(aspect=1)
ggsave('~/Desktop/within-locus_fit.pdf', width=6, height=6)

In [ ]:
R <- 1
fit <- function(N, M) {
    between_cov_fit <- rmsd(ton_covariance_between_loci(s, N, M, R), expected_between_cov_mat)
    return(between_cov_fit)
}
expected_between_cov_mat <- ton_covariance_between_loci(s, true_N, true_M, R)
fit_mat <- apply_grid(fit, n, m)
rownames(fit_mat) <- n
colnames(fit_mat) <- m
plot_fit_mat(fit_mat, labels=FALSE) + geom_point(data = data.frame(n=c(true_N), m=(true_M)), color='red') + labs(y='N', x='N', fill="RMSD") + theme(aspect=1)
ggsave('~/Desktop/between-locus_fit.pdf', width=6, height=6)

In [ ]:
R <- 1
fit <- function(N, M) {
    between_cov_fit <- rmsd(ton_covariance_between_loci(s, N, M, R), expected_between_cov_mat)
    exp_fit <- rmsd(two_locus_expectation(s, N, M, R), expected_exp_mat)
    return(exp_fit + between_cov_fit)
}
expected_between_cov_mat <- ton_covariance_between_loci(s, true_N, true_M, R)
expected_exp_mat <- two_locus_expectation(s, true_N, true_M, R)
fit_mat <- apply_grid(fit, n, m)
rownames(fit_mat) <- n
colnames(fit_mat) <- m
plot_fit_mat(fit_mat, labels=FALSE) + geom_point(data = data.frame(n=c(true_N), m=(true_M)), color='red') + labs(y='N', x='N', fill="RMSD") + theme(aspect=1)
ggsave('~/Desktop/expectation-between-loci_cov_fit.pdf', width=6, height=6)

### expectation

In [ ]:
R <- 1
fit <- function(N, M) {
    exp_fit <- rmsd(two_locus_expectation(s, N, M, R), expected_exp_mat)
    return(exp_fit)
}
expected_exp_mat <- two_locus_expectation(s, true_N, true_M, R)
fit_mat <- apply_grid(fit, n, m)
rownames(fit_mat) <- n
colnames(fit_mat) <- m

plot_fit_mat(fit_mat, labels=FALSE) + geom_point(data = data.frame(n=c(true_N), m=(true_M)), color='red') + labs(y='N', x='N', fill="RMSD") + theme(aspect=1)
ggsave('~/Desktop/expectation_fit.pdf', width=6, height=6)

In [ ]:
R <- 2
fit <- function(N, M) {
    exp_fit <- abs(sum((two_locus_expectation(s, N, M, R) - expected_exp_mat) / expected_exp_mat, na.rm=TRUE))
    return(exp_fit)
}
expected_exp_mat <- two_locus_expectation(s, true_N, true_M, R)
fit_mat <- apply_grid(fit, n, m)
rownames(fit_mat) <- n
colnames(fit_mat) <- m
plot_fit_mat(fit_mat)

### within-locus covariance

In [ ]:
R <- 1
fit <- function(N, M) {
    within_cov_fit <- rmsd(ton_covariance_within_locus(s, N, M, R), expected_within_cov_mat)
    return(within_cov_fit)
}
expected_within_cov_mat <- ton_covariance_within_locus(s, true_N, true_M, R)
fit_mat <- apply_grid(fit, n, m)
rownames(fit_mat) <- n
colnames(fit_mat) <- m
plot_fit_mat(fit_mat, labels=FALSE) + geom_point(data = data.frame(n=c(true_N), m=(true_M)), color='red') + labs(y='N', x='N', fill="RMSD") + theme(aspect=1)
ggsave('~/Desktop/within-locus_fit.pdf', width=6, height=6)

### cross-locus covariance

In [ ]:
R <- 0.1
fit <- function(N, M) {
    between_cov_fit <- rmsd(ton_covariance_between_loci(s, N, M, R), expected_between_cov_mat)
    return(between_cov_fit)
}
expected_between_cov_mat <- ton_covariance_between_loci(s, true_N, true_M, R)
fit_mat <- apply_grid(fit, n, m)
rownames(fit_mat) <- n
colnames(fit_mat) <- m
plot_fit_mat(fit_mat)

In [ ]:
R <- 0.5
fit <- function(N, M) {
    between_cov_fit <- rmsd(ton_covariance_between_loci(s, N, M, R), expected_between_cov_mat)
    return(between_cov_fit)
}
expected_between_cov_mat <- ton_covariance_between_loci(s, true_N, true_M, R)
fit_mat <- apply_grid(fit, n, m)
rownames(fit_mat) <- n
colnames(fit_mat) <- m
plot_fit_mat(fit_mat)

In [ ]:
R <- 1
fit <- function(N, M) {
    between_cov_fit <- rmsd(ton_covariance_between_loci(s, N, M, R), expected_between_cov_mat)
    return(between_cov_fit)
}
expected_between_cov_mat <- ton_covariance_between_loci(s, true_N, true_M, R)
fit_mat <- apply_grid(fit, n, m)
rownames(fit_mat) <- n
colnames(fit_mat) <- m
plot_fit_mat(fit_mat, labels=FALSE) + geom_point(data = data.frame(n=c(true_N), m=(true_M)), color='red') + labs(y='N', x='N', fill="RMSD") + theme(aspect=1)
ggsave('~/Desktop/between-locus_fit.pdf', width=6, height=6)

In [ ]:
fit <- function(N, M) {
    between_cov_fit1 <- rmsd(ton_covariance_between_loci(s, N, M, 0.1), expected_between_cov_mat1)
    between_cov_fit2 <- rmsd(ton_covariance_between_loci(s, N, M, 1), expected_between_cov_mat2)
    return(between_cov_fit1 + between_cov_fit2)
}
expected_between_cov_mat1 <- ton_covariance_between_loci(s, true_N, true_M, 0.1)
expected_between_cov_mat1 <- ton_covariance_between_loci(s, true_N, true_M, 1)
fit_mat <- apply_grid(fit, n, m)
rownames(fit_mat) <- n
colnames(fit_mat) <- m
plot_fit_mat(fit_mat)

In [ ]:
# fit1 <- function(N, M) {
#     between_cov_fit <- rmsd(ton_covariance_between_loci(s, N, M, 0.1), expected_between_cov_mat1)
#     return(between_cov_fit)
# }
# expected_between_cov_mat1 <- ton_covariance_between_loci(s, true_N, true_M, 0.1)
# fit_mat1 <- apply_grid(fit1, n, m)
# rownames(fit_mat1) <- n
# colnames(fit_mat1) <- m

# fit2 <- function(N, M) {
#     between_cov_fit <- rmsd(ton_covariance_between_loci(s, N, M, 1), expected_between_cov_mat)
#     return(between_cov_fit)
# }
# expected_between_cov_mat2 <- ton_covariance_between_loci(s, true_N, true_M, 1)
# fit_mat2 <- apply_grid(fit2, n, m)
# rownames(fit_mat2) <- n
# colnames(fit_mat2) <- m
# fit_mat <- fit_mat1 + fit_mat2
# plot_fit_mat(fit_mat)

In [ ]:
R <- 1
fit <- function(N, M) {
    between_cov_fit <- rmsd(ton_covariance_between_loci(s, N, M, R), expected_between_cov_mat)
    return(between_cov_fit)
}
expected_between_cov_mat <- ton_covariance_between_loci(s, true_N, true_M, R)
fit_mat <- apply_grid(fit, n, m)
rownames(fit_mat) <- n
colnames(fit_mat) <- m
plot_fit_mat(fit_mat, labels=FALSE) + geom_point(data = data.frame(n=c(true_N), m=(true_M)), color='red') + labs(y='N', x='N', fill="RMSD") + theme(aspect=1)
ggsave('~/Desktop/between-cov_fit.pdf', width=6, height=6)

In [ ]:
R <- 10
fit <- function(N, M) {
    between_cov_fit <- rmsd(ton_covariance_between_loci(s, N, M, R), expected_between_cov_mat)
    return(between_cov_fit)
}
expected_between_cov_mat <- ton_covariance_between_loci(s, true_N, true_M, R)
fit_mat <- apply_grid(fit, n, m)
rownames(fit_mat) <- n
colnames(fit_mat) <- m
plot_fit_mat(fit_mat, labels=FALSE) + geom_point(data = data.frame(n=c(true_N), m=(true_M)), color='red') + labs(fill="RMSD")

### expectation, cross-locus covariance

In [ ]:
R <- 0.1
fit <- function(N, M) {
    between_cov_fit <- rsmd(ton_covariance_between_loci(s, N, M, R), expected_between_cov_mat)
    exp_fit <- rsmd(two_locus_expectation(s, N, M, R), expected_exp_mat)
    return(exp_fit + between_cov_fit)
}
expected_between_cov_mat <- ton_covariance_between_loci(s, true_N, true_M, R)
expected_exp_mat <- two_locus_expectation(s, true_N, true_M, R)
fit_mat <- apply_grid(fit, n, m)
rownames(fit_mat) <- n
colnames(fit_mat) <- m
plot_fit_mat(fit_mat)

In [ ]:
R <- 1
fit <- function(N, M) {
    between_cov_fit <- rmsd(ton_covariance_between_loci(s, N, M, R), expected_between_cov_mat)
    exp_fit <- rmsd(two_locus_expectation(s, N, M, R), expected_exp_mat) / expected_exp_mat, na.rm=TRUE))
    return(exp_fit + between_cov_fit)
}
expected_between_cov_mat <- ton_covariance_between_loci(s, true_N, true_M, R)
expected_exp_mat <- two_locus_expectation(s, true_N, true_M, R)
fit_mat <- apply_grid(fit, n, m)
rownames(fit_mat) <- n
colnames(fit_mat) <- m
plot_fit_mat(fit_mat, labels=FALSE) + geom_point(data = data.frame(n=c(true_N), m=(true_M)), color='red') + labs(y='N', x='N', fill="RMSD") + theme(aspect=1)
ggsave('~/Desktop/expectation-between-loci_cov_fit.pdf', width=6, height=6)

In [ ]:
R <- 10
fit <- function(N, M) {
    between_cov_fit <- rmsd(ton_covariance_between_loci(s, N, M, R), expected_between_cov_mat)
    exp_fit <- rmsd(two_locus_expectation(s, N, M, R), expected_exp_mat)
    return(exp_fit + between_cov_fit)
}
expected_between_cov_mat <- ton_covariance_between_loci(s, true_N, true_M, R)
expected_exp_mat <- two_locus_expectation(s, true_N, true_M, R)
fit_mat <- apply_grid(fit, n, m)
rownames(fit_mat) <- n
colnames(fit_mat) <- m
plot_fit_mat(fit_mat)

### Standard optim

In [ ]:
# %%R
# library(stats)

# s <- 4
# # fake empirical observation
# expected_cov_mat <- disc_ton_covariance_between_loci(s, 1, 1, 0.5, 0.5, 1)
# expected_exp_mat <- disc_two_locus_expectation(s, 1, 1, 0.5, 0.5, 1)

# fit <- function(x) {
#     N1 <- x[1]
#     M1 <- x[2]
#     N2 <- N1
#     M2 <- M1
#     # N1 <- x[1]
#     # N2 <- x[2]
#     # M1 <- x[3]
#     # M2 <- x[4]
#     observed_cov_mat <- disc_ton_covariance_between_loci(s, N1, N2, M1, M2, 0.5)
#     observed_exp_mat <- disc_two_locus_expectation(s, N1, N2, M1, M2, 0.5)
#     cov_fit <- abs(sum((observed_cov_mat - expected_cov_mat) / expected_cov_mat, na.rm=TRUE))
#     exp_fit <- abs(sum((observed_exp_mat - expected_exp_mat) / expected_exp_mat, na.rm=TRUE))
#     return(exp_fit + cov_fit)
# }

# outoptim <- optim(c(2, 0.01), fit)
# #outoptim <- optim(c(2, 2, 0.01, 0.01), fit)

In [ ]:
# %%R
# outoptim

### Differential evolution

In [ ]:
# not available for m1 with conda
install.packages("DEoptim")

In [ ]:
library(parallel)
library(foreach)
library(DEoptim)

# s <- 5
# # fake empirical observation
# expected_cov_mat <- disc_ton_covariance_between_loci(s, 1, 0.5, 20)
# expected_exp_mat <- disc_two_locus_expectation(s, 1, 0.5, 20)

# fit <- function(x) {
#     N1 <- x[1]
#     M1 <- x[2]
#     N2 <- N1
#     M2 <- M1
#     observed_cov_mat <- disc_ton_covariance_between_loci(s, N1, N2, M1, M2, 1)
#     observed_exp_mat <- disc_two_locus_expectation(s, N1, N2, M1, M2, 1)
#     cov_fit <- abs(sum((observed_cov_mat - expected_cov_mat) / expected_cov_mat, na.rm=TRUE))
#     exp_fit <- abs(sum((observed_exp_mat - expected_exp_mat) / expected_exp_mat, na.rm=TRUE))
#     return(exp_fit + cov_fit)
# }

fit_wrap <- function(x) {
    N <- x[1]
    M <- x[2]
    return(fit(N, M))
}

controls = DEoptim.control(#itermax = 400,
                           #NP=20, 
                            strategy=1,
                          storepopfrom = 1, 
                          storepopfreq = 1, 
                        parallelType=2)
outDEoptim <- DEoptim(fit_wrap, c(0.5, 0.1), c(1.5, 1.1), controls)
#outDEoptim <- DEoptim(fit_wrap, c(0.5, 0), c(5, 1), controls)
#outDEoptim <- DEoptim(fit_wrap, c(0.5, 0.5, 0.1, 0.1), c(1.5, 1.5, 1.1, 1.1), controls)



In [ ]:
summary(outDEoptim)

In [ ]:
plot(outDEoptim, plot.type="bestmemit")

## Covariance betwen pop1/pop2 descendang configurations at each locus

In [ ]:
p1 <- 0:(s-1)
p2 <- 0:(s-1)
l1 <- expand.grid(p1=p1, p2=p2)
l2 <- expand.grid(p1=p1, p2=p2)

In [ ]:
df <- do.call(cbind.data.frame,Map(expand.grid,l1=l1,l2=l2))
df <- df[order(df$p1.l1, df$p2.l1, df$p1.l2, df$p2.l2),]

In [ ]:
fun <- function(row) {
    covariance(graph, 
                                     rewards[props_to_index(s, row[1], row[2], 1),], 
                                     rewards[props_to_index(s, row[3], row[4], 2),])
}

In [ ]:
m <- matrix(apply(df, 1, fun), c(s**2, s**2))
colnames(m) <- apply(l1, 1, paste, collapse=",")
rownames(m) <- apply(l2, 1, paste, collapse=",")
m

In [ ]:
plot_cov_mat <- function(cov_mat) {
   
    df <- as.data.frame(cov_mat)
    df <- df %>% rownames_to_column('ton1') %>% gather('ton2', 'value', -c(ton1))
    
    ggplot(df, aes(ton1, ton2)) +
        geom_tile(aes(fill = value)) + 
#        scale_y_discrete(labels= seq(1, nrow(cov_mat))) + 
        scale_fill_continuous(type = "viridis") + 
        theme_minimal() + 
        theme(panel.grid.major = element_blank(), panel.grid.minor = element_blank(), text=element_text(size=17)) 

}
plot_cov_mat(m)

## Time-inhomgeneous coalescent

# TODO: 

Implement something like `cdf_timeinhom` from `coalescent-R.ipynb`

# Turn it it an IM model

In [ ]:
# library(ggplot2)
library(viridis)
library(grid)
library(gridExtra)
library(parallel)
library("IRdisplay")
library(tidyverse)
library(devtools)
library(ptdalgorithms)
theme_set(theme_bw())
despine <- theme(panel.border = element_blank(), panel.grid.major = element_blank(),
    panel.grid.minor = element_blank(), axis.line = element_line(colour = "black"),
                text=element_text(size=17)) 
options(repr.plot.width=10, repr.plot.height=7)

source("plot_functions.r")
Rcpp::sourceCpp("./cpp/two_locus_two_island.cpp")
Rcpp::sourceCpp("./cpp/add_epoque.cpp")

In [ ]:
broadcast_props_to_index <- function(s, a, b, p) {
    return((p-1)*(s + 1)**2 + a*(s + 1) + b + 1)
}

two_locus_expectation <- function(s, N, M, R) 
{
    graph <- construct_twolocus_island_graph(s, N, N, M, M, R)
    rewards <- sapply(1:vertices_length(graph), function(index) vertex_at(graph, index)$state )

    exp_mat <- matrix(nrow=s+1,ncol=s+1)
    for (i in 0:s) {
      for (j in 0:s) {
        exp_mat[i+1,j+1] <- expectation(graph, rewards[props_to_index(s, i, j, 1),] + rewards[props_to_index(s, i, j, 2),])
      }
    }
    return(exp_mat)
} 
                      
make_discrete <- function(mutation_graph, mutation_rate) {
    # Takes a graph for a continuous distribution and turns
    # it into a descrete one (inplace). Returns a matrix of
    # rewards for computing marginal moments

    # current nr of states in graph
    vlength <- vertices_length(mutation_graph)

    # number of fields in state vector (assumes all are the same length)
    state_vector_length <- length(vertex_at(mutation_graph, 1)$state)

    # list state vector fields to reward at each auxiliary node
    rewarded_state_vector_indexes <- vector(mode = "list", length = state_vector_length)

    # loop all but starting node
    for (i in 2:vlength) {
        vertex <- vertex_at(mutation_graph,i)
        if (vertex$rate > 0) { # not absorbing
            for (j in 1:length(vertex$state)) {
                val <- vertex$state[j]
                if (val > 0) { # only ones we may reward
                    # add auxilliary node
                    mutation_vertex <- create_vertex(mutation_graph, rep(0, state_vector_length))
                    add_edge(mutation_vertex, vertex, 1)
                    add_edge(vertex, mutation_vertex, mutation_rate*val)

                    rewarded_state_vector_indexes[[mutation_vertex$index]] <- c(rewarded_state_vector_indexes[[j]], j)
                }
            }
        }
    }
    # normalize graph
    weights_were_multiplied_with <- normalize_graph(mutation_graph)

    # build reward matrix
    rewards <- matrix(nrow=vertices_length(mutation_graph),ncol=state_vector_length, 0)
    for (state in seq_along(rewarded_state_vector_indexes)) {
        for (i in rewarded_state_vector_indexes[[state]]) {
            rewards[state, i] <- 1
        }
    }
    rewards = t(rewards)
    return(rewards)
}

# # self-transition rate:
# mutation_rate <- 0.0001

# # clone graph to get one to modify:
# mutation_graph <- clone_graph(graph)

# # add auxilliary states, normalize and return reward matrix:
# rewards <- make_discrete(mutation_graph, mutation_rate)

# # for plotting the new graph
# gam <- graph_as_matrix(mutation_graph)

# vertices_length(mutation_graph)

In [ ]:
add_epoques_to_graph <- function(graph, epoque_params, times) {
  for (i in seq_len(nrow(epoque_params))) {
    add_epoque(graph, epoque_params[i,], times[i])
  }
    
  gam <- graph_as_matrix(graph)
  graph <- matrix_as_graph(gam$IPV, gam$SIM, gam$states)
    
  return(graph)
}

In [ ]:
Sys.getenv('R_MAX_VSIZE')

In [ ]:
#Sys.setenv('R_MAX_VSIZE'=8000000000) 

In [ ]:
inv_N1 <- 1/c(1)
inv_N2 <- 1/c(1)
M1 <- c(0)
M2 <- c(10000)
R <- c(1)
epoque_params <- cbind(inv_N1, inv_N2, M1, M2, R)
times <- c(4)

s <- 5
graph <- construct_twolocus_island_graph(s, 1, 1, 1, 1, 0.5, epoques=TRUE)
graph <- add_epoques_to_graph(graph, epoque_params, times)
rewards <- sapply(1:vertices_length(graph), function(index) head(vertex_at(graph, index)$state, -1))  # need the head to clip off the epoque slot...

mutation_graph <- clone_graph(graph)
print(vertices_length(mutation_graph))
rewards <- make_discrete(mutation_graph, 1)

locus1_rewards <- function(i) {
    colSums(rewards[broadcast_props_to_index(s, i, 1:s, 1),]) + colSums(rewards[broadcast_props_to_index(s, i, 1:s, 2),])
}
locus2_rewards <- function(j) {
    colSums(rewards[broadcast_props_to_index(s, 1:s, j, 1),]) + colSums(rewards[broadcast_props_to_index(s, 1:s, j, 2),])
}

fun <- function(a, b) mapply(function(i, j) covariance(mutation_graph, locus1_rewards(i), locus2_rewards(j)), a, b)
cov_mat <- outer(1:(s-1), 1:(s-1), fun)

plot_cov_mat(cov_mat) + theme(aspect=0.95) + labs(y = '"tons" locus 2', x = '"tons" locus 1', fill='covariance')

In [ ]:
inv_N1 <- 1/c(1)
inv_N2 <- 1/c(1)
M1 <- c(0)
M2 <- c(10000)
R <- c(1)
epoque_params <- cbind(inv_N1, inv_N2, M1, M2, R)
times <- c(4)

s <- 6
graph <- construct_twolocus_island_graph(s, 1, 1, 1, 1, 0.5, epoques=TRUE)
graph <- add_epoques_to_graph(graph, epoque_params, times)
rewards <- sapply(1:vertices_length(graph), function(index) head(vertex_at(graph, index)$state, -1))  # need the head to clip off the epoque slot...

mutation_graph <- clone_graph(graph)
print(vertices_length(mutation_graph))
rewards <- make_discrete(mutation_graph, 1)

locus1_rewards <- function(i) {
    colSums(rewards[broadcast_props_to_index(s, i, 1:s, 1),]) + colSums(rewards[broadcast_props_to_index(s, i, 1:s, 2),])
}
locus2_rewards <- function(j) {
    colSums(rewards[broadcast_props_to_index(s, 1:s, j, 1),]) + colSums(rewards[broadcast_props_to_index(s, 1:s, j, 2),])
}

fun <- function(a, b) mapply(function(i, j) covariance(mutation_graph, locus1_rewards(i), locus2_rewards(j)), a, b)
cov_mat <- outer(1:(s-1), 1:(s-1), fun)

plot_cov_mat(cov_mat) + theme(aspect=0.95) + labs(y = '"tons" locus 2', x = '"tons" locus 1', fill='covariance')

In [ ]:
inv_N1 <- 1/c(1)
inv_N2 <- 1/c(1)
M1 <- c(0)
M2 <- c(10000)
R <- c(1)
epoque_params <- cbind(inv_N1, inv_N2, M1, M2, R)
times <- c(4)

s <- 7
graph <- construct_twolocus_island_graph(s, 1, 1, 1, 1, 0.5, epoques=TRUE)
graph <- add_epoques_to_graph(graph, epoque_params, times)
rewards <- sapply(1:vertices_length(graph), function(index) head(vertex_at(graph, index)$state, -1))  # need the head to clip off the epoque slot...

mutation_graph <- clone_graph(graph)
print(vertices_length(mutation_graph))
rewards <- make_discrete(mutation_graph, 1)

locus1_rewards <- function(i) {
    colSums(rewards[broadcast_props_to_index(s, i, 1:s, 1),]) + colSums(rewards[broadcast_props_to_index(s, i, 1:s, 2),])
}
locus2_rewards <- function(j) {
    colSums(rewards[broadcast_props_to_index(s, 1:s, j, 1),]) + colSums(rewards[broadcast_props_to_index(s, 1:s, j, 2),])
}

fun <- function(a, b) mapply(function(i, j) covariance(mutation_graph, locus1_rewards(i), locus2_rewards(j)), a, b)
cov_mat <- outer(1:(s-1), 1:(s-1), fun)

plot_cov_mat(cov_mat) + theme(aspect=0.95) + labs(y = '"tons" locus 2', x = '"tons" locus 1', fill='covariance')

In [9]:

# self-transition rate:
mutation_rate <- 5e-10 * 20000 * 30

disc_two_locus_expectation <- function(s, N, M, R, T) 
{    
    graph <- construct_twolocus_island_graph(s, N, N, M, M, R, epoques=TRUE)
    inv_N1 <- 1/c(N)
    inv_N2 <- 1/c(N)
    M1 <- c(0)
    M2 <- c(10000)
    R <- c(R)
    epoque_params <- cbind(inv_N1, inv_N2, M1, M2, R)
    times <- c(T)
    graph <- add_epoques_to_graph(graph, epoque_params, times)

    mutation_graph <- clone_graph(graph)
    rewards <- make_discrete(mutation_graph, mutation_rate)
    
    exp_mat <- matrix(nrow=s-1,ncol=s-1)
    for (i in 1:(s-1)) {
      for (j in 1:(s-1)) {
        exp_mat[i,j] <- expectation(mutation_graph, rewards[props_to_index(s, i, j, 1),] + rewards[props_to_index(s, i, j, 2),])
      }
    }
    return(exp_mat)
} 

disc_ton_covariance_within_locus <- function(s, N, M, R, T) 
{
    graph <- construct_twolocus_island_graph(s, N, N, M, M, R, epoques=TRUE)
    inv_N1 <- 1/c(N)
    inv_N2 <- 1/c(N)
    M1 <- c(0)
    M2 <- c(10000)
    R <- c(R)
    epoque_params <- cbind(inv_N1, inv_N2, M1, M2, R)
    times <- c(T)
    graph <- add_epoques_to_graph(graph, epoque_params, times)
    
    mutation_graph <- clone_graph(graph)
    rewards <- make_discrete(mutation_graph, mutation_rate)
    
    locus1_rewards <- function(i) {
        colSums(rewards[broadcast_props_to_index(s, i, 1:s, 1),]) + colSums(rewards[broadcast_props_to_index(s, i, 1:s, 2),])
    }

    fun <- function(a, b) mapply(function(i, j) covariance(mutation_graph, locus1_rewards(i), locus1_rewards(j)), a, b)
    cov_mat <- outer(1:(s-1), 1:(s-1), fun)
    return(cov_mat)
}  

disc_ton_covariance_between_loci <- function(s, N, M, R, T) 
{
    graph <- construct_twolocus_island_graph(s, N, N, M, M, R, epoques=TRUE)
    inv_N1 <- 1/c(N)
    inv_N2 <- 1/c(N)
    M1 <- c(0)
    M2 <- c(10000)
    R <- c(R)
    epoque_params <- cbind(inv_N1, inv_N2, M1, M2, R)
    times <- c(T)
    graph <- add_epoques_to_graph(graph, epoque_params, times)
    
    mutation_graph <- clone_graph(graph)
    rewards <- make_discrete(mutation_graph, 1)
    
    locus1_rewards <- function(i) {
        colSums(rewards[broadcast_props_to_index(s, i, 1:s, 1),]) + colSums(rewards[broadcast_props_to_index(s, i, 1:s, 2),])
    }
    locus2_rewards <- function(j) {
        colSums(rewards[broadcast_props_to_index(s, 1:s, j, 1),]) + colSums(rewards[broadcast_props_to_index(s, 1:s, j, 2),])
    }

    fun <- function(a, b) mapply(function(i, j) covariance(mutation_graph, locus1_rewards(i), locus2_rewards(j)), a, b)
    cov_mat <- outer(1:(s-1), 1:(s-1), fun)
    return(cov_mat)
} 

In [10]:
mat <- disc_two_locus_expectation(5, 1, 1, 0.5, 4)
plot_exp_mat(mat) +
scale_x_discrete(labels= seq(0, nrow(exp_mat))) + 
scale_y_discrete(labels= seq(0, nrow(exp_mat))) + 
scale_fill_viridis() +
theme_minimal() +
theme(aspect=1) + labs(y = "locus 2", x = "locus 1", fill='expectation') +
 theme(panel.grid.major = element_blank(), 
        panel.grid.minor = element_blank(), 
        text=element_text(size=17))

ggsave('~/Desktop/tmp.pdf', width=6, height=6)

ERROR: Error in make_discrete(mutation_graph, mutation_rate): could not find function "make_discrete"


----